<!-- Developed by Renan Amaral de Oliveira - https://www.linkedin.com/in/renanammaral/ -->
# <font color='#415a77'>**Assessing Turnaround Process Bottlenecks to Optimize Airport Operations**</font>
## <font color = '#415a77'> **Developed by <a href="https://www.linkedin.com/in/renanammaral/">Renan Amaral de Oliveira**</a></font>
### <font color='#415a77'> **Identifying the Variables That Most Affect the Total Turnaround Time**</font>

<font color='#415a77'>

# **The Problem:**

One of the major challenges faced by airports and airlines is the optimization of operations to ensure smooth air traffic, reduce costs, and improve the passenger experience. One of the key performance indicators in this context is the turnaround time.


### **What is Turnaround?**

Turnaround is one of the most critical processes in airport operations, as it directly affects the efficiency, punctuality, and profitability of airlines. This term refers to the time required for an aircraft to complete all ground activities between landing and the next takeoff. The faster and more efficient this process is, the lower the operational costs and the greater the airport’s capacity to handle a higher volume of flights.


### **The Importance of Turnaround:**

Optimizing turnaround reduces delays, improves punctuality, and increases operational efficiency, positively impacting the reputation of both airlines and airports. Moreover, a well-managed turnaround enables better fleet utilization, contributing to the sustainability of the aviation sector.


### **The Challenge:**

In this scenario, there is a need to develop an analytical model capable of identifying which processes may be creating operational bottlenecks and affecting the total ground time, considering various factors such as:

	- Passenger disembarkation/embarkation
	- Baggage removal
	- Aircraft cleaning
	- Restocking of supplies
	- Fuel refueling
	- Maintenance and technical inspections
	- Baggage and cargo loading
	- Passenger boarding
	- Crew change

### **Benefits of Optimizing Turnaround Time:**

	- Delay reduction: With a faster and more efficient operational flow, flights depart on schedule, minimizing impacts on the flight network.
	- Increased operational capacity: Optimized processes allow more flights to be handled without the need for infrastructure expansion.
	- Reduced operational costs: Less ground time means lower expenses with fuel, staff, and airport fees.
	- Improved passenger experience: Faster boarding and disembarkation lead to greater satisfaction and comfort for travelers.
	- Sustainability: Less ground waiting time results in lower carbon emissions, contributing to environmental efficiency.
	- Fleet optimization: With less idle time, aircraft can perform more flights per day, maximizing profitability.
	- Greater predictability: With predictive analysis tools, it’s possible to anticipate and address bottlenecks before they impact operations.

### **Next Steps:**

To solve this problem, the following steps are necessary:

	- Data collection and organization: Gather historical data on airport operations, including records of each process that affects turnaround.
	- Exploratory data analysis: Identify the most important variables and their relationships with turnaround time.
	- Building a predictive model: Use machine learning techniques to develop a model capable of identifying the processes that most affect turnaround time.
	- Model validation: Assess the model’s accuracy using a test dataset.
	- Implementation and monitoring: Integrate the model into a decision-support system and continuously monitor its performance.
	- By implementing a solution to predict taxi time, airports and airlines can optimize their operations, reduce costs, and enhance customer satisfaction.

</font>

# Initial Setup

## Imports

In [1]:
# importing the necessary libraries
%pip install ydata-profiling
%pip install -q -U watermark

# importing data manipulation libraries
import pandas as pd
import numpy as np

#importing os manipulation libraries
import os

# importing visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# importing machine learning libraries
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# importing machine learning models
from sklearn.linear_model import LinearRegression, LogisticRegression
import xgboost as xgb

# importing evaluation metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error, r2_score

# importing profiling library
from ydata_profiling import ProfileReport

#importing warnings library
import warnings
warnings.filterwarnings('ignore')

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%reload_ext watermark
%watermark -a "Renan Amaral" -u -d -v -p pandas,n

Author: Renan Amaral

Last updated: 2025-07-08

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.27.0

pandas: 2.2.2
n     : not installed



# Loading and visualizing data

In [3]:
# loading the dataset in the folder 'data/raw'
#print(os.getcwd())
df = pd.read_csv('/Users/renanoliveira/Documents/Portfolio/turnaround/data/raw/turnaround_data_100K.csv')

In [4]:
df.head()

,flight_number,airline,origin,destination,arrival_time,inblock_time,deboarding_start,cleaning_start,refueling_start,boarding_start,door_close_time,offblock_time,takeoff_time,turnaround_minutes,delay_minutes
0,LA1630,Latam,REC,CGH,2025-04-29 07:00:00,2025-04-29 07:03:00,2025-04-29 07:09:00,2025-04-29 07:14:00,2025-04-29 07:20:00,2025-04-29 07:33:00,2025-04-29 07:42:00,2025-04-29 07:43:00,2025-04-29 07:53:00,40,8
1,AZ7696,Azul,FOR,GRU,2025-01-04 07:55:00,2025-01-04 07:56:00,2025-01-04 08:06:00,2025-01-04 08:14:00,2025-01-04 08:24:00,2025-01-04 08:39:00,2025-01-04 08:45:00,2025-01-04 08:48:00,2025-01-04 08:54:00,52,14
2,AZ7124,Azul,CWB,CNF,2025-05-01 09:28:00,2025-05-01 09:29:00,2025-05-01 09:39:00,2025-05-01 09:44:00,2025-05-01 09:54:00,2025-05-01 10:00:00,2025-05-01 10:10:00,2025-05-01 10:11:00,2025-05-01 10:21:00,42,8
3,AZ5411,Azul,CNF,SSA,2025-04-09 10:43:00,2025-04-09 10:46:00,2025-04-09 10:52:00,2025-04-09 11:00:00,2025-04-09 11:10:00,2025-04-09 11:22:00,2025-04-09 11:32:00,2025-04-09 11:34:00,2025-04-09 11:41:00,48,13
4,LA3823,Latam,FOR,GRU,2025-05-09 22:40:00,2025-05-09 22:41:00,2025-05-09 22:47:00,2025-05-09 22:54:00,2025-05-09 23:03:00,2025-05-09 23:11:00,2025-05-09 23:21:00,2025-05-09 23:22:00,2025-05-09 23:37:00,41,12
